In [1]:
import pandas as pd
import re

In [2]:
# LOAD csv
file_path = '../data/curated/properties_with_closest_shopping_centers.csv'  
df = pd.read_csv(file_path)


In [3]:
columns_to_drop = ['Closest Station', 'Closest School', 'Closest Shopping Center']
df_cleaned = df.drop(columns=columns_to_drop)

In [4]:
df_cleaned

,Property Name,Property Latitude,Property Longitude,Distance (km),cost_text,School Distance (km),SA2_NAME21,Distance to Closest Shopping Center (km)
0,"6/137 Coppin St, Richmond VIC 3121",-37.820283,145.003572,0.882026,$425 weekly,0.326233,Richmond (South) - Cremorne,9.543373
1,"205/6 Joseph Road, Footscray VIC 3011",-37.798238,144.914181,0.990486,$510 per week,0.723669,Footscray,3.431348
2,"8/7 Docker St, Richmond VIC 3121",-37.823681,144.997096,0.308573,$425 weekly,0.676342,Richmond (South) - Cremorne,10.037550
3,"8/51 Kooyong Road, Armadale VIC 3143",-37.862342,145.019539,0.658460,$375 per week,0.917637,Armadale,6.446263
4,"10/54 Rockley Road, South Yarra VIC 3141",-37.835164,145.000360,0.785119,$475 pw,0.360197,South Yarra - North,9.419477
...,...,...,...,...,...,...,...,...
3078,"176 Scotchmans Rd, Drysdale VIC 3222",-38.149923,144.626622,23.251456,"$1,800 weekly",4.556564,Portarlington,29.994924
3079,"485 Punt Road, South Yarra VIC 3141",-37.830086,144.987847,0.699074,"$1,450.00",0.321831,South Yarra - North,10.630510
3080,"4 Vital drive, Tarneit VIC 3029",-37.858304,144.647505,3.788183,$550,1.152332,Tarneit (West) - Mount Cottrell,5.051280
3081,"14 Sheffield Way, Keysborough VIC 3173",-38.000807,145.149657,4.452369,$625 / wk,0.542452,Keysborough - South,13.587161


In [5]:
# Define function to clean
def clean_rent_price(cost_text):
    match = re.search(r'(\d+[,.]?\d*)', cost_text.replace(',', ''))
    if match:
        return float(match.group(1))
    else:
        return None

df_cleaned['weekly_rent'] = df_cleaned['cost_text'].apply(clean_rent_price)

print(df_cleaned[['cost_text', 'weekly_rent']].head())

       cost_text  weekly_rent
0    $425 weekly        425.0
1  $510 per week        510.0
2    $425 weekly        425.0
3  $375 per week        375.0
4        $475 pw        475.0


In [6]:

Q1 = df_cleaned['weekly_rent'].quantile(0.25)
Q3 = df_cleaned['weekly_rent'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_outliers = df_cleaned[(df_cleaned['weekly_rent'] >= lower_bound) & (df_cleaned['weekly_rent'] <= upper_bound)]

print(df_no_outliers[['cost_text', 'weekly_rent']].head())


       cost_text  weekly_rent
0    $425 weekly        425.0
1  $510 per week        510.0
2    $425 weekly        425.0
3  $375 per week        375.0
4        $475 pw        475.0


In [7]:
max_values = df_no_outliers.max()

min_values = df_no_outliers.min()

print("Max values for each column:\n", max_values)
print("\nMin values for each column:\n", min_values)


Max values for each column:
 Property Name                               Park Orchards VIC 3114
Property Latitude                                       -34.182998
Property Longitude                                      147.646661
Distance (km)                                           183.163879
cost_text                                      from $300 per night
School Distance (km)                                     12.134035
SA2_NAME21                                              Yarraville
Distance to Closest Shopping Center (km)                455.890734
weekly_rent                                                 1200.0
dtype: object

Min values for each column:
 Property Name                               1 & 2/23 Koonawarra Street, Clayton VIC 3168
Property Latitude                                                             -38.771375
Property Longitude                                                            142.115746
Distance (km)                                            

In [8]:
df_no_outliers

,Property Name,Property Latitude,Property Longitude,Distance (km),cost_text,School Distance (km),SA2_NAME21,Distance to Closest Shopping Center (km),weekly_rent
0,"6/137 Coppin St, Richmond VIC 3121",-37.820283,145.003572,0.882026,$425 weekly,0.326233,Richmond (South) - Cremorne,9.543373,425.0
1,"205/6 Joseph Road, Footscray VIC 3011",-37.798238,144.914181,0.990486,$510 per week,0.723669,Footscray,3.431348,510.0
2,"8/7 Docker St, Richmond VIC 3121",-37.823681,144.997096,0.308573,$425 weekly,0.676342,Richmond (South) - Cremorne,10.037550,425.0
3,"8/51 Kooyong Road, Armadale VIC 3143",-37.862342,145.019539,0.658460,$375 per week,0.917637,Armadale,6.446263,375.0
4,"10/54 Rockley Road, South Yarra VIC 3141",-37.835164,145.000360,0.785119,$475 pw,0.360197,South Yarra - North,9.419477,475.0
...,...,...,...,...,...,...,...,...,...
3076,"13 Adelaide St, Blairgowrie VIC 3942",-38.367311,144.785258,34.419625,$850 weekly,3.431329,Point Nepean,54.130591,850.0
3077,"160 Moore rd, Sunbury VIC 3429",-37.608981,144.711509,2.216868,$1100 Per Week,1.528344,Sunbury - South,11.078158,1100.0
3080,"4 Vital drive, Tarneit VIC 3029",-37.858304,144.647505,3.788183,$550,1.152332,Tarneit (West) - Mount Cottrell,5.051280,550.0
3081,"14 Sheffield Way, Keysborough VIC 3173",-38.000807,145.149657,4.452369,$625 / wk,0.542452,Keysborough - South,13.587161,625.0


In [9]:

df_no_outliers = df_no_outliers.drop(columns=['cost_text'])


df_no_outliers = df_no_outliers.rename(columns={'weekly_rent': 'cost_text'})
df_no_outliers

,Property Name,Property Latitude,Property Longitude,Distance (km),School Distance (km),SA2_NAME21,Distance to Closest Shopping Center (km),cost_text
0,"6/137 Coppin St, Richmond VIC 3121",-37.820283,145.003572,0.882026,0.326233,Richmond (South) - Cremorne,9.543373,425.0
1,"205/6 Joseph Road, Footscray VIC 3011",-37.798238,144.914181,0.990486,0.723669,Footscray,3.431348,510.0
2,"8/7 Docker St, Richmond VIC 3121",-37.823681,144.997096,0.308573,0.676342,Richmond (South) - Cremorne,10.037550,425.0
3,"8/51 Kooyong Road, Armadale VIC 3143",-37.862342,145.019539,0.658460,0.917637,Armadale,6.446263,375.0
4,"10/54 Rockley Road, South Yarra VIC 3141",-37.835164,145.000360,0.785119,0.360197,South Yarra - North,9.419477,475.0
...,...,...,...,...,...,...,...,...
3076,"13 Adelaide St, Blairgowrie VIC 3942",-38.367311,144.785258,34.419625,3.431329,Point Nepean,54.130591,850.0
3077,"160 Moore rd, Sunbury VIC 3429",-37.608981,144.711509,2.216868,1.528344,Sunbury - South,11.078158,1100.0
3080,"4 Vital drive, Tarneit VIC 3029",-37.858304,144.647505,3.788183,1.152332,Tarneit (West) - Mount Cottrell,5.051280,550.0
3081,"14 Sheffield Way, Keysborough VIC 3173",-38.000807,145.149657,4.452369,0.542452,Keysborough - South,13.587161,625.0


In [10]:
df_no_outliers = df_no_outliers.rename(columns={'Distance (km)': 'Station Distance (km)'})

df_no_outliers = df_no_outliers.rename(columns={'cost_text': 'rent price'})

df_no_outliers

,Property Name,Property Latitude,Property Longitude,Station Distance (km),School Distance (km),SA2_NAME21,Distance to Closest Shopping Center (km),rent price
0,"6/137 Coppin St, Richmond VIC 3121",-37.820283,145.003572,0.882026,0.326233,Richmond (South) - Cremorne,9.543373,425.0
1,"205/6 Joseph Road, Footscray VIC 3011",-37.798238,144.914181,0.990486,0.723669,Footscray,3.431348,510.0
2,"8/7 Docker St, Richmond VIC 3121",-37.823681,144.997096,0.308573,0.676342,Richmond (South) - Cremorne,10.037550,425.0
3,"8/51 Kooyong Road, Armadale VIC 3143",-37.862342,145.019539,0.658460,0.917637,Armadale,6.446263,375.0
4,"10/54 Rockley Road, South Yarra VIC 3141",-37.835164,145.000360,0.785119,0.360197,South Yarra - North,9.419477,475.0
...,...,...,...,...,...,...,...,...
3076,"13 Adelaide St, Blairgowrie VIC 3942",-38.367311,144.785258,34.419625,3.431329,Point Nepean,54.130591,850.0
3077,"160 Moore rd, Sunbury VIC 3429",-37.608981,144.711509,2.216868,1.528344,Sunbury - South,11.078158,1100.0
3080,"4 Vital drive, Tarneit VIC 3029",-37.858304,144.647505,3.788183,1.152332,Tarneit (West) - Mount Cottrell,5.051280,550.0
3081,"14 Sheffield Way, Keysborough VIC 3173",-38.000807,145.149657,4.452369,0.542452,Keysborough - South,13.587161,625.0


In [11]:

file_path2 = '../data/curated/increase_rates.csv' 
df2 = pd.read_csv(file_path2)
desired_columns = [
    'District', 'Mar 2017 to Jun 2017 % Increase', 'Jun 2017 to Sep 2017 % Increase', 
    'Sep 2017 to Dec 2017 % Increase', 'Dec 2017 to Mar 2018 % Increase', 
    'Mar 2018 to Jun 2018 % Increase', 'Jun 2018 to Sep 2018 % Increase', 
    'Sep 2018 to Dec 2018 % Increase', 'Dec 2018 to Mar 2019 % Increase', 
    'Mar 2019 to Jun 2019 % Increase', 'Jun 2019 to Sep 2019 % Increase', 
    'Sep 2019 to Dec 2019 % Increase', 'Dec 2019 to Mar 2020 % Increase', 
    'Mar 2020 to Jun 2020 % Increase', 'Jun 2020 to Sep 2020 % Increase', 
    'Sep 2020 to Dec 2020 % Increase', 'Dec 2020 to Mar 2021 % Increase', 
    'Mar 2021 to Jun 2021 % Increase', 'Jun 2021 to Sep 2021 % Increase', 
    'Sep 2021 to Dec 2021 % Increase', 'Dec 2021 to Mar 2022 % Increase', 
    'Mar 2022 to Jun 2022 % Increase', 'Jun 2022 to Sep 2022 % Increase', 
    'Sep 2022 to Dec 2022 % Increase', 'Dec 2022 to Mar 2023 % Increase'
]

df2_selected = df2[desired_columns]

df2_selected

,District,Mar 2017 to Jun 2017 % Increase,Jun 2017 to Sep 2017 % Increase,Sep 2017 to Dec 2017 % Increase,Dec 2017 to Mar 2018 % Increase,Mar 2018 to Jun 2018 % Increase,Jun 2018 to Sep 2018 % Increase,Sep 2018 to Dec 2018 % Increase,Dec 2018 to Mar 2019 % Increase,Mar 2019 to Jun 2019 % Increase,...,Sep 2020 to Dec 2020 % Increase,Dec 2020 to Mar 2021 % Increase,Mar 2021 to Jun 2021 % Increase,Jun 2021 to Sep 2021 % Increase,Sep 2021 to Dec 2021 % Increase,Dec 2021 to Mar 2022 % Increase,Mar 2022 to Jun 2022 % Increase,Jun 2022 to Sep 2022 % Increase,Sep 2022 to Dec 2022 % Increase,Dec 2022 to Mar 2023 % Increase
0,Albert Park,3.000000,-2.912621,5.000000,-4.761905,3.000000,-2.912621,5.000000,-4.761905,3.000000,...,5.000000,-4.761905,3.000000,-2.912621,5.000000,-4.761905,3.000000,-2.912621,5.000000,3.809524
1,Middle Park,3.000000,-2.912621,5.000000,-4.761905,3.000000,-2.912621,5.000000,-4.761905,3.000000,...,5.000000,-4.761905,3.000000,-2.912621,5.000000,-4.761905,3.000000,-2.912621,5.000000,3.809524
2,West St Kilda,3.000000,-2.912621,5.000000,-4.761905,3.000000,-2.912621,5.000000,-4.761905,3.000000,...,5.000000,-4.761905,3.000000,-2.912621,5.000000,-4.761905,3.000000,-2.912621,5.000000,3.809524
3,Armadale,4.651163,0.000000,2.222222,-6.521739,4.651163,0.000000,2.222222,-6.521739,4.651163,...,2.222222,-6.521739,4.651163,0.000000,2.222222,-6.521739,4.651163,0.000000,2.222222,6.521739
4,Carlton North,2.586207,0.840336,0.000000,-3.333333,2.586207,0.840336,0.000000,-3.333333,2.586207,...,0.000000,-3.333333,2.586207,0.840336,0.000000,-3.333333,2.586207,0.840336,0.000000,3.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Traralgon,5.555556,0.000000,0.000000,-5.263158,5.555556,0.000000,0.000000,-5.263158,5.555556,...,0.000000,-5.263158,5.555556,0.000000,0.000000,-5.263158,5.555556,0.000000,0.000000,1.315789
211,Wanagaratta,2.702703,0.000000,0.000000,-2.631579,2.702703,0.000000,0.000000,-2.631579,2.702703,...,0.000000,-2.631579,2.702703,0.000000,0.000000,-2.631579,2.702703,0.000000,0.000000,0.000000
212,Warragul,0.000000,5.000000,2.380952,-6.976744,0.000000,5.000000,2.380952,-6.976744,0.000000,...,2.380952,-6.976744,0.000000,5.000000,2.380952,-6.976744,0.000000,5.000000,2.380952,2.325581
213,Warrnambool,0.000000,5.000000,0.000000,-4.761905,0.000000,5.000000,0.000000,-4.761905,0.000000,...,0.000000,-4.761905,0.000000,5.000000,0.000000,-4.761905,0.000000,5.000000,0.000000,0.000000


In [12]:
output_file_path = '../data/curated/selected_columns_file.csv' 
df2_selected.to_csv(output_file_path, index=False)

print("文件已成功清理并保存为:", output_file_path)

文件已成功清理并保存为: ../data/curated/selected_columns_file.csv


In [13]:
# LOAD csv
file_path3 = '../data/curated/cleaned_data_2.csv'  
df3 = pd.read_csv(file_path3)

In [14]:
df3 = df3.rename(columns={'name': 'Property Name'}).drop(columns=['rent'])

df3


,Property Name,bedrooms,bathrooms,parking_spots
0,"17/25 Williams Road, Prahran VIC 3181",1,1,0
1,"6/137 Coppin St, Richmond VIC 3121",1,1,0
2,"410/378-390 St Georges Road, Fitzroy North VIC...",1,1,1
3,"205/6 Joseph Road, Footscray VIC 3011",1,1,1
4,"8/7 Docker St, Richmond VIC 3121",1,1,1
...,...,...,...,...
4259,"13 Adelaide St, Blairgowrie VIC 3942",5,3,1
4260,"160 Moore rd, Sunbury VIC 3429",5,2,0
4261,"4 Vital drive, Tarneit VIC 3029",5,2,1
4262,"14 Sheffield Way, Keysborough VIC 3173",5,2,1


In [15]:
# needed row
df3_subset = df3[['Property Name', 'bedrooms', 'bathrooms', 'parking_spots']]


df_cleaned = df_no_outliers.merge(df3_subset, on='Property Name', how='left')


df_cleaned


,Property Name,Property Latitude,Property Longitude,Station Distance (km),School Distance (km),SA2_NAME21,Distance to Closest Shopping Center (km),rent price,bedrooms,bathrooms,parking_spots
0,"6/137 Coppin St, Richmond VIC 3121",-37.820283,145.003572,0.882026,0.326233,Richmond (South) - Cremorne,9.543373,425.0,1.0,1.0,0.0
1,"205/6 Joseph Road, Footscray VIC 3011",-37.798238,144.914181,0.990486,0.723669,Footscray,3.431348,510.0,1.0,1.0,1.0
2,"8/7 Docker St, Richmond VIC 3121",-37.823681,144.997096,0.308573,0.676342,Richmond (South) - Cremorne,10.037550,425.0,1.0,1.0,1.0
3,"8/51 Kooyong Road, Armadale VIC 3143",-37.862342,145.019539,0.658460,0.917637,Armadale,6.446263,375.0,1.0,1.0,1.0
4,"10/54 Rockley Road, South Yarra VIC 3141",-37.835164,145.000360,0.785119,0.360197,South Yarra - North,9.419477,475.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2847,"13 Adelaide St, Blairgowrie VIC 3942",-38.367311,144.785258,34.419625,3.431329,Point Nepean,54.130591,850.0,5.0,3.0,1.0
2848,"160 Moore rd, Sunbury VIC 3429",-37.608981,144.711509,2.216868,1.528344,Sunbury - South,11.078158,1100.0,5.0,2.0,0.0
2849,"4 Vital drive, Tarneit VIC 3029",-37.858304,144.647505,3.788183,1.152332,Tarneit (West) - Mount Cottrell,5.051280,550.0,5.0,2.0,1.0
2850,"14 Sheffield Way, Keysborough VIC 3173",-38.000807,145.149657,4.452369,0.542452,Keysborough - South,13.587161,625.0,5.0,2.0,1.0


In [16]:
df_cleaned = df_cleaned.dropna(subset=['bathrooms'])

df_cleaned


,Property Name,Property Latitude,Property Longitude,Station Distance (km),School Distance (km),SA2_NAME21,Distance to Closest Shopping Center (km),rent price,bedrooms,bathrooms,parking_spots
0,"6/137 Coppin St, Richmond VIC 3121",-37.820283,145.003572,0.882026,0.326233,Richmond (South) - Cremorne,9.543373,425.0,1.0,1.0,0.0
1,"205/6 Joseph Road, Footscray VIC 3011",-37.798238,144.914181,0.990486,0.723669,Footscray,3.431348,510.0,1.0,1.0,1.0
2,"8/7 Docker St, Richmond VIC 3121",-37.823681,144.997096,0.308573,0.676342,Richmond (South) - Cremorne,10.037550,425.0,1.0,1.0,1.0
3,"8/51 Kooyong Road, Armadale VIC 3143",-37.862342,145.019539,0.658460,0.917637,Armadale,6.446263,375.0,1.0,1.0,1.0
4,"10/54 Rockley Road, South Yarra VIC 3141",-37.835164,145.000360,0.785119,0.360197,South Yarra - North,9.419477,475.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2847,"13 Adelaide St, Blairgowrie VIC 3942",-38.367311,144.785258,34.419625,3.431329,Point Nepean,54.130591,850.0,5.0,3.0,1.0
2848,"160 Moore rd, Sunbury VIC 3429",-37.608981,144.711509,2.216868,1.528344,Sunbury - South,11.078158,1100.0,5.0,2.0,0.0
2849,"4 Vital drive, Tarneit VIC 3029",-37.858304,144.647505,3.788183,1.152332,Tarneit (West) - Mount Cottrell,5.051280,550.0,5.0,2.0,1.0
2850,"14 Sheffield Way, Keysborough VIC 3173",-38.000807,145.149657,4.452369,0.542452,Keysborough - South,13.587161,625.0,5.0,2.0,1.0


In [17]:

output_file_path = '../data/curated/df_cleaned.csv'
df_cleaned.to_csv(output_file_path, index=False)

print("文件已成功清理并保存为:", output_file_path)

文件已成功清理并保存为: ../data/curated/df_cleaned.csv
